# Experiment Env. for Signal DIE Prototype CTM on RHSC-ET Thermal Analysis #

In [1]:
#### import packages ####
import os
import ipywidgets as widgets
from ipywidgets import Label, HBox, VBox
from ipyfilechooser import FileChooser

#### import MY packages ####
from parsers import RHSCETparser
from setups import FCpowermap
from setups import utilities

style = {"description_width": "initial"}

In [27]:
### Create CASE ###
CASE = widgets.Text(value="TEST", description="CASE Name:", disabled=False)
PWfc = FileChooser("./")   ### power file generated from FC
MDfc = FileChooser("./")
TFfc = FileChooser("./")
PWType = widgets.RadioButtons(options=["IPF","CSV","TILE"], description="Power file type", disabled=False)
HB1 = HBox([Label("Select Power File"), PWfc, PWType])
HB2 = HBox([Label("Select Metal Density File"), MDfc])
HB3 = HBox([Label("Select Totem TechFile"), TFfc])
VBox([CASE, HB1, HB2, HB3])
#display(PWfc)

In [28]:
### Create case in CASEs folder base on CASE Name ###
#import importlib
#importlib.reload(utilities)
root = os.path.join(os.getcwd(), "CASEs")
caseFolder = utilities.createCASE(root, CASE.value)
caseDB = os.path.join(caseFolder, "DB")

In [29]:
### Optional: Manually given the design area ###
isManually = widgets.Select(options=["True", "False"], value="False", description="Design Area in Manually", disabled=False)
llx = widgets.FloatText(value=0.0, description="LLX", disabled=False)
lly = widgets.FloatText(value=0.0, description="LLY", disabled=False)
urx = widgets.FloatText(value=0.0, description="URX", disabled=False)
ury = widgets.FloatText(value=0.0, description="URY", disabled=False)
HB1 = HBox([llx, lly, urx, ury])
VBox([isManually, HB1])

In [31]:
### Exam Power Map ###
import importlib
importlib.reload(FCpowermap)

designArea = []
try:
    if isManually.value in ["True"]:
        designArea = [llx.value, lly.value, urx.value, ury.value]
except:
    pass

print("Design Area: {}".format(designArea))
FCpwrView = FCpowermap.FCPowerView(PWfc.selected, PWType.value, DesignArea=designArea)

Design Area: [59.0, 23.0, 196.0, 219.0]
<INFO> Instances Area:59.016,23.616,195.528,218.736; Total PWR=5.342mW


In [32]:
### Create Prototype CTM ###
designArea, ETmhsPath = FCpwrView.RHSCETmhs(outputFolder=caseDB)
CTM_name = widgets.Text(value="CTM01", description="CTM Name", disabled=False)
CTM_resolution = widgets.IntText(value=1, description="CTM Resolution", disabled=False)
HB1 = HBox([CTM_resolution, Label("um")])
VBox([CTM_name, HB1])

In [33]:
#print(designArea, PWfc.selected, MDfc.selected, TFfc.selected, CTM_resolution.value)
DIE_X = "{:.3f}".format(designArea[2]-designArea[0])
DIE_Y = "{:.3f}".format(designArea[3]-designArea[1])
CTM_r = str(CTM_resolution.value)

### make a copy to the case DB folder ###
os.system("cp -rf {} {}".format(PWfc.selected, caseDB))
os.system("cp -rf {} {}".format(MDfc.selected, caseDB))
os.system("cp -rf  {} {}".format(TFfc.selected, caseDB))
CTMparaDict = {"RUN_DIR":caseFolder, "CTMNAME":CTM_name.value, "DIE_LENGTH":DIE_X, "DIE_WIDTH":DIE_Y, \
               "RESOLUTION":CTM_r, "PWRMAP":ETmhsPath, "DENSITY":MDfc.selected, "TOTEM_TF":TFfc.selected}

templateScript = os.path.join(os.path.join(os.getcwd(), "ET_scripts"), "prototypeCTM.tcl")
ETscript = utilities.createETscript(CTMparaDict, template=templateScript, outputFolder=caseDB, outputName="genPrototypeCTM.tcl")

os.system("rm -rf {}".format(os.path.join(caseFolder, "*CTM*")))
runcmd = ["#!/bin/csh",\
          "source /nfs/site/disks/x5e2d_gwa_chienchi_001/Tools/licenses/ansys.csh", \
          "redhawk_sc_et -3dic -ng {}".format(ETscript)]
runcshPath = os.path.join(caseDB, "genCTM.csh")

progBar = widgets.IntProgress(max=100)
display(progBar)
CTMpath = os.path.join(os.path.join(CTMparaDict["RUN_DIR"], "CTM"), CTMparaDict["CTMNAME"]+".tar.gz")
utilities.submitJob(runcmd, runcshPath=runcshPath, progressBar=progBar, checkFiles=[CTMpath])

IntProgress(value=0)

Your job has been queued (JobID 2217339209, Class SLES15SP4, Queue zsc9_dts, Slot /de/pdk/dev/sl_int)


True

In [34]:
### Select the Thermal TechFile ###
TTFfc = FileChooser("./")
HBox([Label("Select Thermal TF"), TTFfc])

In [35]:
import importlib
from parsers import ThermalTFparser
importlib.reload(ThermalTFparser) 

TTF = ThermalTFparser.parseThermalTF(TTFfc.selected)

## TTF.TTFNames
TTFNames = ["p1278_m14_BULK", "p1278_m14_woBULK", "p1278_m18_BULK", "p1280_m16_BULK", "p1280_m16_woBULK"]

### Setup DIE thermal simulation inputs ###
ETCaseName = widgets.Text(value="DEMO", description="ET Project Name", style=style, disabled=False)
DIE_H = widgets.FloatText(value=0.0, description="DIE height", style=style, disabled=False)
TFDIE = widgets.Dropdown(options=TTFNames, description="TTF DIE Label", style=style, disabled=False)
totalPWR = widgets.FloatText(value=1.0, description="Total PWR", style=style, disabled=False)
deltaT = widgets.FloatText(value=10.0, description="delta T", style=style, disabled=False)
ambT = widgets.FloatText(value=25.0, description="Ambient T", style=style, disabled=False)
TOPBC = widgets.FloatText(value=1000.0, description="Top DIE BC", style=style, disabled=False)
BOTBC = widgets.FloatText(value=1000.0, description="Bot DIE BC", style=style, disabled=False)
HB1 = HBox([DIE_H, Label("um")])
HB2 = HBox([TFDIE])
HB3 = HBox([totalPWR, Label("mW"), deltaT])
HB4 = HBox([ambT, TOPBC, Label("H"), BOTBC, Label("H")])
VBox([ETCaseName, HB1, HB2, HB3, HB4])

In [37]:
### simple HTC estimation: to refine the HTC values ###
refHTC = utilities.simpleCalHTC(totalPWR.value, designArea, deltaT=deltaT.value)
#refHTC = utilities.simpleCalHTC(1.3, [0,0,135,135], 2.4)
print("Estimated HTC: {}".format(refHTC))

Estimated HTC: 3289.6


In [38]:
### Create ET script ###
DIEparaDict = {"RUN_DIR":caseFolder, "NAME":ETCaseName.value, "CTM":CTMpath, "DIE_LENGTH":DIE_X, \
               "DIE_WIDTH":DIE_Y, "DIE_HEIGHT":str(DIE_H.value), "TTF":TTFfc.selected, "TFDIE":TFDIE.value, \
               "AMBIENT_T":str(ambT.value), "TOPBC":str(TOPBC.value), "BOTBC":str(BOTBC.value) }

templateScript = os.path.join(os.path.join(os.getcwd(), "ET_scripts"), "signalDIE.tcl")
ETscript = utilities.createETscript(DIEparaDict, template=templateScript, outputFolder=caseDB, outputName="DEMO_et.tcl")

ETFolder = os.path.join(caseFolder, ETCaseName.value+"_et")
os.system("rm -rf {}".format(ETFolder))
runcmd = ["#!/bin/csh",\
          "source /nfs/site/disks/x5e2d_gwa_chienchi_001/Tools/licenses/ansys.csh", \
          "redhawk_sc_et -3dic -ng {}".format(ETscript)]
runcshPath = os.path.join(caseDB, "runET.csh")

progBar = widgets.IntProgress(max=100)
display(progBar)
TProfile = os.path.join(ETFolder, "ThermalProfile_DIE.txt")
checkFiles = [os.path.join(ETFolder, "analysis/STATIC_RUN/thermal/STATIC_RUN.dbg"), \
              TProfile]
utilities.submitJob(runcmd, runcshPath=runcshPath, progressBar=progBar, checkFiles=checkFiles)

IntProgress(value=0)

Your job has been queued (JobID 2217345300, Class SLES15SP4, Queue zsc9_dts, Slot /de/pdk/dev/sl_int)


True

In [ ]:
### analysis ET Thermal Profile ###
ETview = RHSCETparser.RHSCETView(TProfile)
ETview.plot(isShow=True)

## UT1. Open the RHSC-ET ##
Create a folder "OpenET" for execution

In [1]:
### Optional: Submit job to open RHSC_ET GUI mode ###
import os
import importlib
from setups import utilities
importlib.reload(utilities)

root_ut1 = os.path.join(os.getcwd(), "CASEs")
_caseFolder = utilities.createCASE(root_ut1, "OpenET")
_caseDB = os.path.join(_caseFolder, "DB")

runcmd = ["#!/bin/csh",
          "source /nfs/site/disks/x5e2d_gwa_chienchi_001/Tools/licenses/ansys.csh", \
          "redhawk_sc_et -3dic"]
runcshPath = os.path.join(_caseDB, "runETGUI.csh")
utilities.submitJob(runcmd, runcshPath=runcshPath)

Your job has been queued (JobID 2300631647, Class SLES15SP4, Queue zsc9_dts, Slot /de/pdk/dev/sl_int)


True